Import all the packages

In [1]:
!pip install stanza
!pip install textblob

     |████████████████████████████████| 432 kB 5.1 MB/s 
     |████████████████████████████████| 170 kB 48.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=39a5ba5ca0d22640a0514d0e17eba223f267d94fafd8e61f658990f9d68d93d1
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [2]:
!pip install langdetect

     |████████████████████████████████| 981 kB 5.2 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=5d0534967018720dad428366adffaa01ccac732ceb7c815e4bedc0b63ceebd7e
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [3]:
import pandas as pd
import numpy as np
import re
import stanza
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import nltk
import langdetect
from langdetect import detect
from textblob import TextBlob as tb

In [4]:
stanza.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

2021-12-06 16:12:07 INFO: Downloading default packages for language: en (English)...


2021-12-06 16:12:22 INFO: Finished downloading models and saved to /root/stanza_resources.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Import dataset

In [5]:
#df=pd.read_csv('garnierEnComBV.csv')

In [20]:
df = pd.read_csv('vichy_reviews_amazon.csv')
df.rename(columns={'Review':'review_text'},inplace=True)

In [21]:
df

,ASIN,ProductName,Stars,ReviewTitle,review_text
0,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",5.0,Best hyaluronic acid,I'm an avid skin care fanatic and have tried s...
1,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",5.0,Definitely worth it,At first I was skeptical but I've seen this re...
2,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",5.0,Light Weight,"This moisturizer is light weight, non-sticky/n..."
3,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",2.0,Dried my skin,"Unfortunately, it dried my skin and created dr..."
4,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",5.0,Worth it!,I have been wanting to try Vichy for a long ti...
...,...,...,...,...,...
14833,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",NaN,Maria Cordoneda Sánchez,Mi madre quedo encantada con el producto. Te d...
14834,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",NaN,Una piccola delusione,"Sono fan dei prodotti di Vichy da anni, uso da..."
14835,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",NaN,Buon prodotto penso lo ricomprerò,Mi piace molto perché anche a fine giornata si...
14836,B06XC5PQGL,"Vichy Hydrating Hyaluronic Acid Serum, Mineral...",NaN,Buona,Crema perfetta x l'estate ha la consistenza di...


In [8]:
nlp = stanza.Pipeline()

def preprocess(review):
      sentence = review
      sentence = re.sub('[^a-zA-Z]',' ',sentence)
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
      sentence = re.sub(r'\s+', ' ',sentence)

      return sentence

2021-12-06 16:12:23 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2021-12-06 16:12:23 INFO: Use device: gpu
2021-12-06 16:12:23 INFO: Loading: tokenize
2021-12-06 16:12:32 INFO: Loading: pos
2021-12-06 16:12:33 INFO: Loading: lemma
2021-12-06 16:12:33 INFO: Loading: depparse
2021-12-06 16:12:33 INFO: Loading: sentiment
2021-12-06 16:12:34 INFO: Loading: constituency
2021-12-06 16:12:34 INFO: Loading: ner
2021-12-06 16:12:35 INFO: Done loading processors!


In [9]:
len(df)

31484

In [18]:
# Python3 program to count words
# in a given string
OUT = 0
IN = 1
 
# Returns number of words in string
def countWords(string):
    state = OUT
    wc = 0
 
    # Scan all characters one by one
    for i in range(len(string)):
 
        # If next character is a separator,
        # set the state as OUT
        if (string[i] == ' ' or string[i] == '\n' or
            string[i] == '\t'):
            state = OUT
 
        # If next character is not a word
        # separator and state is OUT, then
        # set the state as IN and increment
        # word count
        elif state == OUT:
            state = IN
            wc += 1
 
    # Return the number of words
    return wc
 
# Driver Code
string = "One two         three\n four\tfive "
print("No. of words : " + str(countWords(string)))

No. of words : 5


In [22]:
aspect = []
lang = []
sentiment = []
for i in range(len(df)):
  txt = df['review_text'][i]
  txt=txt.lower()
  #print(i)
  #print('Number of Words:',countWords(txt))
  #Create a def to remove all the upper lower case, punctuations, and extra white space
  

  def preprocess(review):
    sentence = review
    sentence = re.sub('[^a-zA-Z]',' ',sentence)
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
    sentence = re.sub(r'\s+', ' ',sentence)

    return sentence

  temp=preprocess(txt)
  sentList=nltk.sent_tokenize(temp)

  if countWords(temp)== 0 or countWords(temp)== 1:
    aspect.append(txt)
    sentiment.append(tb(txt).sentiment)
    continue

  lang.append(detect(txt))
  if detect(txt)!= 'en':
    sentiment.append('')
    continue
  #Word tokenize and Part-of-speech Tag each word

  for line in sentList:
      txt_list = nltk.word_tokenize(line)
      taggedList = nltk.pos_tag(txt_list)
  #|print(taggedList)

  #Join words with two nouns together(E.g. hair product = hairproduct)then pos tag everything again

  newwordList = []
  flag = 0
  for i in range(0,len(taggedList)-1):
      if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
          newwordList.append(taggedList[i][0]+taggedList[i+1][0])
          flag=1
      else:
          if(flag==1):
              flag=0
              continue
          newwordList.append(taggedList[i][0])
          if(i==len(taggedList)-2):
              newwordList.append(taggedList[i+1][0])
  finaltxt = ' '.join(word for word in newwordList)

  sentList = nltk.sent_tokenize(finaltxt)
  for line in sentList:
      txt_list = nltk.word_tokenize(line)
      taggedList = nltk.pos_tag(txt_list)
  #print('taggedlist1',taggedList)

  #remove any stop words
  stop_words = set(stopwords.words('english'))
  new_txt_list = nltk.word_tokenize(finaltxt)
  wordsList = [w for w in new_txt_list if not w in stop_words]
  taggedList = nltk.pos_tag(wordsList)
  
  #exception for GW,which is causing the model to not run 
  i=0
  while i<len(taggedList):
      if taggedList[i][1]=='GW':
          #print('yes')
          finaltxt.replace(taggedList[i][0],' ')
          taggedList.pop(i)
          continue
      i+=1

  finaltxt2=[]
  for i in range(0,len(taggedList)):
    finaltxt2.append(taggedList[i][0])
  finaltxt = ' '.join(word for word in finaltxt2)

  if finaltxt == []:
    continue
  #Use a constituency parser to determine the relationship between each word
  doc = nlp(finaltxt)
  dep_node = []

  for dep_edge in doc.sentences[0].dependencies:
    dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
  for i in range(0, len(dep_node)):
    if (int(dep_node[i][1]) != 0):
      dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]

  #print('dep_node',dep_node)
  featureList = []
  categories = []
  totalfeatureList = []
  #print('taggedlist',taggedList)
  for i in taggedList:
    if(i[1]=='JJ' or i[1]=='JJS' or i[1]=='JJR' or i[1]=='NN' or i[1]=='NNS' or i[1]=='NNP' or i[1]=='NNPS' or i[1]=='RB'):
      featureList.append(list(i))
      totalfeatureList.append(list(i)) # This list will store all the features for every sentence
      categories.append(i[0])
  #print('featurelist',featureList)
  #print('categories:',categories)
  #possible relations: "obj',"neg", "prep_of",, "compound"
  fcluster = []
  for i in featureList:
    filist = []
    for j in dep_node:
      if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "prep_of", "acomp", "xcomp"])):
        if(j[0]==i[0]):
          filist.append(j[1])
        else:
          filist.append(j[0])
    fcluster.append([i[0], filist])
  
  #print('fcluster',fcluster)
  finalcluster = []
  dic = {}
  for i in totalfeatureList:
      dic[i[0]] = i[1]
  for i in fcluster:
      if(dic[i[0]]=="NN" or dic[i[0]]=="NNS" or dic[i[0]]=="NNP" or dic[i[0]]=="NNPS"):
          finalcluster.append(i)

  i=0
  while i < len(finalcluster):
    if bool(finalcluster[i][1])==False:
      finalcluster.pop(i)
      continue
    i+=1
  #print('Finalcluster',finalcluster)
  aspect.append(finalcluster)
  sentiment.append(tb(finaltxt).sentiment)
  #print('sentiment:', sentiment)
df['cluster']=aspect
df['language']=lang
df['sentiment']=sentiment
    

2021-12-06 20:06:12 ERROR: Went infinite!:
Final state:
State(
  buffer:[(NN ampoulepack), (VB see), (NN day), (NN pack), (JJ latter), (RB fairly), (VBN priced), (VBP continue), (VBG starting), (MD would), (VB spring), (JJR larger), (VB pack), (JJS least), (JJ full), (NN month), (VB determine), (JJ good), (JJ enough), (NNS results), (VB continue), (VBN said), (NN combination), (NNS ingredients), (VBP seem), (JJ beneficial), (NN love), (VBG using), (NN vitamin), (NN brightening), (JJ hyaluronic), (NN acid), (NN humectant), (NN waterretention), (NN addition), (NN plant), (VBN based), (NNS peptides), (NNS phytopeptides), (JJ riceprotein), (NN lupine), (NN proteinclaim), (NN help), (NN skin), (NN firmness), (RB already), (VBP use), (JJ first), (CD two), (VBD tried), (JJ latter), (NN routine), (NN plant), (VBN based), (NN sense), (RB typically), (VB use), (NN animal), (VBN sourced), (VBN hydrolyzed), (NN protein), (NNS peptides), (NNS partdon), (VB expect), (VB see), (JJ much), (NN change),

AssertionError: ignored

In [ ]:
df['review_text'][2]

In [12]:
df

,review_id,review_text,review_rating,review_title
0,196991408,Premi¨¨re fois que j'ai essay¨¦ le 50 j'ai tou...,5.0,Premi¨¨re fois que j'ai essay¨¦ le 50 j'ai tou...
1,197033183,I leave this conditioner in my hair for up to ...,4.0,I leave this conditioner in my hair for up to ...
2,229195055,OBSESSED!! makes me glow to the gods! So smoot...,5.0,OBSESSED!! makes me glow
3,229199240,I¡¯ve been using this serum for a few days now...,5.0,I love it!
4,229200919,It was a bit thinner than I anticipated but I ...,3.0,Good
...,...,...,...,...
31479,221480246,This leave in added so much nourishment and so...,5.0,so nourishing!!
31480,221646682,"My daughter has super curly hair, when she nap...",5.0,curly hair
31481,221677448,Whole Blends Castor Oil Leave-In Treatment fro...,2.0,Too much weigh down
31482,221699589,The texture is thick and creamy. I used it mos...,4.0,"Not bad, give it a try."
